In [48]:
from modules.datakit import *
from openai import OpenAI
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [2]:
path = f'data_processed/main.csv'
df = pd.read_csv(path)

df = deduplicate_main(df)
df = transform_data(
    main = df,
    only_expired = True,
    duration_start = 1,
    duration_end = 28,
    utilize_morf = False
)

In [3]:
df = df[df.added_dt.le('2025-02-04')&df.additional_fees.lt(100)].copy()

In [4]:
df = df[['link', 'rent', 'additional_fees', 'adv_description']].copy()

In [5]:
pattern = r'(\d{1,3}(?:[.,]\d{3})+|\d{3,})'

def filter_lines(text):

    lines = text.split('\n')
    filtered = [line for line in lines if re.search(pattern, line)]
    return "\n".join(filtered)

df['filtered_description'] = df['adv_description'].apply(filter_lines)

In [6]:
df = df[df.filtered_description.ne('')].reset_index(drop=True).copy()

__----__

In [8]:
client = OpenAI(api_key=open('api_key.txt', 'r').read())

In [9]:
prompt = """Wyodrębnij z poniższego opisu ogłoszenia wszystkie dodatkowe opłaty (np. czynsz administracyjny, zaliczki na media) wraz z ich wartościami.
Odpowiedz tylko w formacie: "nazwa opłaty: wartość".
Nie uwzględniaj kaucji. Nie używaj separatorów tysięcy.
Jeśli nie ma dodatkowych opłat, napisz "Brak dodatkowych opłat.".
Tekst ogłoszenia: {}
"""

In [10]:
def query_llm(text, prompt):
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "Jesteś asystentem do ekstrakcji informacji z tekstu."},
            {"role": "user", "content": prompt.format(text)}
        ],
        temperature=0,
        max_tokens=200
    )

    return completion.choices[0].message.content

In [11]:
df['llm_output'] = df['filtered_description'].apply(lambda x: query_llm(x, prompt))

In [25]:
pattern = r'(\d{3,})'
def extract_and_sum(text, pattern):
    matches = re.findall(pattern, text)
    total = 0
    for match in matches:
        total += int(match)
    return total

In [27]:
df['real_additional_fees'] = df['llm_output'].apply(lambda x: extract_and_sum(x, pattern))

In [36]:
path = 'data_raw\large_language_model/llm_output.csv'
df.to_csv(path,
          encoding='utf-8',
          index=False)